In [1]:
import olefile

In [38]:
## 이건 한두페이지밖에 추출이 안됨
f = olefile.OleFileIO('./test_folder/「국방 지식관리시스템 운영 예규」 개정본문.hwp')
encoded_text = f.openstream('PrvText').read()
decoded_text = encoded_text.decode('utf-16')
# decoded_text

'  제정 국방부예규 제 478호(’10. 4. 13.)\r\n일부개정 국방부예규 제 513호(’12. 5. 22.)\r\n일부개정 국방부예규 제 573호(’16. 2. 22.)\r\n일부개정 국방부예규 제 591호(’17. 6. 13.)\r\n일부개정 국방부예규 제621호(’20. 11. 4.)\r\n\r\n   \r\n\r\n<국방 지식관리시스템 운영 예규>\r\n\r\n  목      차\r\n\r\n제 1 장  총  칙\r\n제 1 조    목    적  1\r\n제 2 조    정    의  1\r\n제 3 조    적용범위  2\r\n\r\n제 2 장  지식관리체계 및 절차\r\n제 4 조    지식정보책임관  3\r\n제 5 조    지식정보관리자  3\r\n제 6 조    지식정보담당자  3\r\n제 8 조    시스템관리  3\r\n제 9 조    등록  3\r\n제 10 조   평가  4\r\n제 11 조   지식의 관리  4\r\n제 12 조   지식관리운영위원회의 구성  5\r\n제 13 조   지식관리운영위원회의 운영  5\r\n제 3 장  학습동아리(CoP)활동\r\n제 14 조  구성  6\r\n제 15 조  운영  6\r\n제 16 조  평가 및 지원  7\r\n\r\n제 4 장  지식마일리지제도 및 보상\r\n제 17 조  지식마일리지제도 운영  7\r\n제 18 조  우수지식 등록자 선정 및 보상  8\r\n제 19 조  지식활동의 활성화  8\r\n제 20 조  지식관리의 날 지정 운영  9\r\n\r\n부칙  9\r\n별표  11\r\n별지  12\r\n\r\n\r\n국방 지식관리시스템 운영 예규\r\n\r\n   제1장 총 칙\r\n제1조(목적) 이 예규는 국방부 지식관리시스템(KMS)의 운영 및 관리에 관하여 필요한 사항을 규정하고, 지식관리시스템을 이용한 조직의 지식경쟁력을 통하여 조직 활동성과를 높이기 위함을 목적으로 한다.\r\n제2조(정의) 이 예규에서 사용하는 용어의 정의는 다음 각 호와 같다.\r\n  1. “

In [43]:
import olefile
import zlib
import struct

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data
    
        # 각 Section 내 text 추출    
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67]:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                section_text += rec_data.decode('utf-16')
                section_text += "\n"
                # print(section_text)

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text
# [출처] [Python] 각종 문서에서 텍스트 추출하기|작성자 IML

In [44]:
text = get_hwp_text('./test_folder/「국방 지식관리시스템 운영 예규」 개정본문.hwp')
with open('./txt_folder/text.txt', 'w', encoding='utf-16') as text_file:
    text_file.write(text)

In [7]:

!pip install --user --pre pyhwp

     |████████████████████████████████| 225kB 10.3MB/s eta 0:00:01
  Created wheel for pyhwp: filename=pyhwp-0.1b15-cp37-none-any.whl size=315441 sha256=3eeb3e4bb35bd83f2eac239eeef0319016c6b04dbef50e2d70e00a1956d67ffe
  Stored in directory: /home/minds/.cache/pip/wheels/56/d7/7f/050d0b98805dada5b9f8ea498352ff5dc192f2be92422b8ccd
Successfully built pyhwp


In [10]:
## 터미널에서 사용하기
!hwp5txt --output './test_folder/text.txt' '「국방 지식관리시스템 운영 예규」 개정본문.hwp'

In [21]:
import os
# import win32com.client as win32

import_path = './test_folder/' ## hwp 파일 저장된 경로 (폴더)
convert_path = './txt_folder' ## text 파일 저장되어야 하는 경로 (폴더)
exefile = 'hwp5txt'

## 한글파일 txt로 변환해서 저장

res = []
for root, dirs, files in os.walk(import_path):
    # rootpath = os.path.join(os.path.abspath(import_path), root)
    import_rootpath = os.path.abspath(import_path)
    convert_rootpath = os.path.abspath(convert_path)

    for file in files:
        import_file_path = os.path.join(import_rootpath, file)
        convert_file_path = os.path.join(convert_rootpath, file[:-4]+".txt")
        output = '--output ' + '"' + convert_file_path + '"'
        result = '"' + import_file_path + '"'
        print(exefile + ' ' + output + ' ' + result)
        os.system(exefile + ' ' + output + ' ' + result)